In [1]:
!pip install bnlp-toolkit -q
!pip install banglanum2words -q
!pip install git+https://github.com/csebuetnlp/normalizer -q
!pip install bnunicodenormalizer -q
!pip install bnlp-toolkit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.10 requires scipy>=1.11.2, but you have scipy 1.10.1 which is incompatible.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but y

In [2]:
import re
import os
import time
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import unicodedata
from normalizer import normalize
from banglanum2words import num_convert

from bnunicodenormalizer import Normalizer 
from pprint import pprint
bnorm=Normalizer()

from bnlp import NLTKTokenizer
bnltk = NLTKTokenizer()

In [3]:
df = pd.read_csv('/kaggle/input/normalization-quality-testing/test.csv')
df

,file_name,transcripts,district
0,test_sandwip (1).wav,"হুম, আই দাওয়াত খাইবো। খাইবো অনকা তো বুজি <> হব...",sandwip
1,test_sandwip (2).wav,"এন্নে বিয়া-সাদি চুয়াইছে, ঘরে সংসারও চইলছে, হুম...",sandwip
2,test_sandwip (3).wav,"হুম, হাক্কন হিডা ন নি হারা যাইবো নে? এক্কবারে ...",sandwip
3,test_sandwip (4).wav,"আম-কাডল <> ধর ইয়া আছে, হল আছে তারফরে দুধ আছে আ...",sandwip
4,test_sandwip (5).wav,এই তুই গোসল কইত্তি ন? হিয়া হরে কইরবো এরি। গোসল...,sandwip
...,...,...,...
1695,test_sylhet (364).wav,"তার বাদে ব্যাবসা-বানিজ্য, টাখা ফয়সার লেন-দেন, ...",sylhet
1696,test_sylhet (365).wav,"তে দেখা গেছে এই দিকটার মানুষরা এরা, এরার বড়ো শ...",sylhet
1697,test_sylhet (366).wav,"যারা এই ডকুমেন্ট <> দেখায়, এরা খয় যো এরার, এরা...",sylhet
1698,test_sylhet (367).wav,"তারা মনে খরে যে ওইটা, ওইটা আমার রুট। এইযে রুট ...",sylhet


In [4]:
df['sen_len'] = df['transcripts'].apply(lambda x: len(x.split()))
df

,file_name,transcripts,district,sen_len
0,test_sandwip (1).wav,"হুম, আই দাওয়াত খাইবো। খাইবো অনকা তো বুজি <> হব...",sandwip,31
1,test_sandwip (2).wav,"এন্নে বিয়া-সাদি চুয়াইছে, ঘরে সংসারও চইলছে, হুম...",sandwip,26
2,test_sandwip (3).wav,"হুম, হাক্কন হিডা ন নি হারা যাইবো নে? এক্কবারে ...",sandwip,36
3,test_sandwip (4).wav,"আম-কাডল <> ধর ইয়া আছে, হল আছে তারফরে দুধ আছে আ...",sandwip,29
4,test_sandwip (5).wav,এই তুই গোসল কইত্তি ন? হিয়া হরে কইরবো এরি। গোসল...,sandwip,25
...,...,...,...,...
1695,test_sylhet (364).wav,"তার বাদে ব্যাবসা-বানিজ্য, টাখা ফয়সার লেন-দেন, ...",sylhet,43
1696,test_sylhet (365).wav,"তে দেখা গেছে এই দিকটার মানুষরা এরা, এরার বড়ো শ...",sylhet,41
1697,test_sylhet (366).wav,"যারা এই ডকুমেন্ট <> দেখায়, এরা খয় যো এরার, এরা...",sylhet,43
1698,test_sylhet (367).wav,"তারা মনে খরে যে ওইটা, ওইটা আমার রুট। এইযে রুট ...",sylhet,30


In [5]:
df[df['sen_len']==0]

,file_name,transcripts,district,sen_len


# Checking the data types present in the dataframe

In [6]:
types = {}

for i in df['transcripts']:
    t = type(i)
    if t not in types:
        types[t] = 1
    else:
        types[t] += 1

types

{str: 1700}

In [7]:
null_ind = []

for index, value in df['transcripts'].items():
    if type(value) == float:
        null_ind.append(index)
        #print(f"Index: {index}, Value: {value}")

print()
print(null_ind)


[]


In [8]:
df.iloc[null_ind]

,file_name,transcripts,district,sen_len


In [9]:
null_mask = df.isnull().any(axis=1)
null_rows = df[null_mask]

null_rows

,file_name,transcripts,district,sen_len


# Checking for numerical texts (Both Bangla and English)

In [10]:
num_df = df.copy()
num_df

,file_name,transcripts,district,sen_len
0,test_sandwip (1).wav,"হুম, আই দাওয়াত খাইবো। খাইবো অনকা তো বুজি <> হব...",sandwip,31
1,test_sandwip (2).wav,"এন্নে বিয়া-সাদি চুয়াইছে, ঘরে সংসারও চইলছে, হুম...",sandwip,26
2,test_sandwip (3).wav,"হুম, হাক্কন হিডা ন নি হারা যাইবো নে? এক্কবারে ...",sandwip,36
3,test_sandwip (4).wav,"আম-কাডল <> ধর ইয়া আছে, হল আছে তারফরে দুধ আছে আ...",sandwip,29
4,test_sandwip (5).wav,এই তুই গোসল কইত্তি ন? হিয়া হরে কইরবো এরি। গোসল...,sandwip,25
...,...,...,...,...
1695,test_sylhet (364).wav,"তার বাদে ব্যাবসা-বানিজ্য, টাখা ফয়সার লেন-দেন, ...",sylhet,43
1696,test_sylhet (365).wav,"তে দেখা গেছে এই দিকটার মানুষরা এরা, এরার বড়ো শ...",sylhet,41
1697,test_sylhet (366).wav,"যারা এই ডকুমেন্ট <> দেখায়, এরা খয় যো এরার, এরা...",sylhet,43
1698,test_sylhet (367).wav,"তারা মনে খরে যে ওইটা, ওইটা আমার রুট। এইযে রুট ...",sylhet,30


In [11]:
num_ind = []

pattern = re.compile('[0-9০-৯]')

for index, value in num_df['transcripts'].items():

    if pattern.search(str(value)):
        if type(value) == float or value == "Null":
            pass
        else:
            num_ind.append(index)

print()
print(num_ind)


[]


In [12]:
num_df = df.iloc[num_ind]
num_df

,file_name,transcripts,district,sen_len


# Checking for English alphabets in texts

In [13]:
eng_df = df.copy()
eng_df

,file_name,transcripts,district,sen_len
0,test_sandwip (1).wav,"হুম, আই দাওয়াত খাইবো। খাইবো অনকা তো বুজি <> হব...",sandwip,31
1,test_sandwip (2).wav,"এন্নে বিয়া-সাদি চুয়াইছে, ঘরে সংসারও চইলছে, হুম...",sandwip,26
2,test_sandwip (3).wav,"হুম, হাক্কন হিডা ন নি হারা যাইবো নে? এক্কবারে ...",sandwip,36
3,test_sandwip (4).wav,"আম-কাডল <> ধর ইয়া আছে, হল আছে তারফরে দুধ আছে আ...",sandwip,29
4,test_sandwip (5).wav,এই তুই গোসল কইত্তি ন? হিয়া হরে কইরবো এরি। গোসল...,sandwip,25
...,...,...,...,...
1695,test_sylhet (364).wav,"তার বাদে ব্যাবসা-বানিজ্য, টাখা ফয়সার লেন-দেন, ...",sylhet,43
1696,test_sylhet (365).wav,"তে দেখা গেছে এই দিকটার মানুষরা এরা, এরার বড়ো শ...",sylhet,41
1697,test_sylhet (366).wav,"যারা এই ডকুমেন্ট <> দেখায়, এরা খয় যো এরার, এরা...",sylhet,43
1698,test_sylhet (367).wav,"তারা মনে খরে যে ওইটা, ওইটা আমার রুট। এইযে রুট ...",sylhet,30


In [14]:
eng_ind = []

pattern = re.compile('[a-zA-Z]')

for index, value in eng_df['transcripts'].items():

    if pattern.search(str(value)):
        if type(value) == float or value == "Null":
            pass
        else:
            eng_ind.append(index)

print()
print(eng_ind)


[362]


In [15]:
eng_df = df.iloc[eng_ind]
eng_df

,file_name,transcripts,district,sen_len
362,test_chittagong (125).wav,"ande ugga family foirge, tinjon, maa maiyafua ...",chittagong,30


# Replacing any newline characters or any other irrelevant characters

In [16]:
# print(list(df[df['transcripts'] == ''].index))
df.drop(df[df['transcripts'] == ''].index, inplace=True)

# print(list(df[df['transcripts'] == '<>'].index))
df.drop(df[df['transcripts'] == "<>"].index, inplace=True)
      
# print(list(df[df['transcripts'] == '..'].index))
df.drop(df[df['transcripts'] == ".."].index, inplace=True)

# print(list(df[df['transcripts'] == '..'].index))
df.drop(df[df['transcripts'] == "."].index, inplace=True)

In [17]:
df[df["transcripts"] == "<>"]

,file_name,transcripts,district,sen_len


In [18]:
df[df["transcripts"] == ""]

,file_name,transcripts,district,sen_len


In [19]:
df[df["transcripts"] == "."]

,file_name,transcripts,district,sen_len


In [20]:
df[df["transcripts"] == ".."]

,file_name,transcripts,district,sen_len


In [21]:
transcripts = []

for i in range(len(df)):
    transcript = df["transcripts"].iloc[i]
    transcript = transcript.replace("\n", " ")
    transcript = " ".join(transcript.split())
    transcripts.append(transcript)

df['transcripts'] = transcripts

df

,file_name,transcripts,district,sen_len
0,test_sandwip (1).wav,"হুম, আই দাওয়াত খাইবো। খাইবো অনকা তো বুজি <> হব...",sandwip,31
1,test_sandwip (2).wav,"এন্নে বিয়া-সাদি চুয়াইছে, ঘরে সংসারও চইলছে, হুম...",sandwip,26
2,test_sandwip (3).wav,"হুম, হাক্কন হিডা ন নি হারা যাইবো নে? এক্কবারে ...",sandwip,36
3,test_sandwip (4).wav,"আম-কাডল <> ধর ইয়া আছে, হল আছে তারফরে দুধ আছে আ...",sandwip,29
4,test_sandwip (5).wav,এই তুই গোসল কইত্তি ন? হিয়া হরে কইরবো এরি। গোসল...,sandwip,25
...,...,...,...,...
1695,test_sylhet (364).wav,"তার বাদে ব্যাবসা-বানিজ্য, টাখা ফয়সার লেন-দেন, ...",sylhet,43
1696,test_sylhet (365).wav,"তে দেখা গেছে এই দিকটার মানুষরা এরা, এরার বড়ো শ...",sylhet,41
1697,test_sylhet (366).wav,"যারা এই ডকুমেন্ট <> দেখায়, এরা খয় যো এরার, এরা...",sylhet,43
1698,test_sylhet (367).wav,"তারা মনে খরে যে ওইটা, ওইটা আমার রুট। এইযে রুট ...",sylhet,30


In [22]:
df["transcripts"].str.len().idxmin(), df["transcripts"].str.len().idxmax(), df["transcripts"].str.len().min(), df["transcripts"].str.len().max()

(442, 576, 8, 503)

In [23]:
df["transcripts"].info()

<class 'pandas.core.series.Series'>
Index: 1698 entries, 0 to 1699
Series name: transcripts
Non-Null Count  Dtype 
--------------  ----- 
1698 non-null   object
dtypes: object(1)
memory usage: 26.5+ KB


In [24]:
df["transcripts"].isna().sum()

0

# Final Processing

In [25]:
def normalize_text(text):
    # Tokenize the text into words
    words = text.split()    
    # Normalize each word
    normalized_words = [bnorm(word)["normalized"] for word in words if isinstance(word, str)]
    # Reconstruct the text with normalized words
    normalized_text = ' '.join(word for word in normalized_words if word is not None)
    return normalized_text

def unicode_to_ascii(s):
    return unicodedata.normalize('NFKC', s)

def remove_extra_whitespace(sentence):
    words = sentence.split()
    return " ".join(words)

def preprocess_sentence(text,remove_english=True):
    text = text.lower().strip()
    text = normalize_text(text)
    text = unicode_to_ascii(text)
    text = remove_extra_whitespace(text)
    return text

In [26]:
df = df.copy()
df["transcripts"] = df["transcripts"].apply(preprocess_sentence)

df

,file_name,transcripts,district,sen_len
0,test_sandwip (1).wav,"হুম, আই দাওয়াত খাইবো। খাইবো অনকা তো বুজি <> হ...",sandwip,31
1,test_sandwip (2).wav,"এন্নে বিয়া-সাদি চুয়াইছে, ঘরে সংসারও চইলছে, হ...",sandwip,26
2,test_sandwip (3).wav,"হুম, হাক্কন হিডা ন নি হারা যাইবো নে? এক্কবারে ...",sandwip,36
3,test_sandwip (4).wav,"আম-কাডল <> ধর ইয়া আছে, হল আছে তারফরে দুধ আছে ...",sandwip,29
4,test_sandwip (5).wav,এই তুই গোসল কইত্তি ন? হিয়া হরে কইরবো এরি। গোস...,sandwip,25
...,...,...,...,...
1695,test_sylhet (364).wav,"তার বাদে ব্যাবসা-বানিজ্য, টাখা ফয়সার লেন-দেন,...",sylhet,43
1696,test_sylhet (365).wav,"তে দেখা গেছে এই দিকটার মানুষরা এরা, এরার বড়ো ...",sylhet,41
1697,test_sylhet (366).wav,"যারা এই ডকুমেন্ট <> দেখায়, এরা খয় যো এরার, এ...",sylhet,43
1698,test_sylhet (367).wav,"তারা মনে খরে যে ওইটা, ওইটা আমার রুট। এইযে রুট ...",sylhet,30
